In [1]:
# Generate Random Latitudes and Longitudes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
# add citipy library
from citipy import citipy

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)


lat_lngs = zip(lats, lngs)
lat_lngs


# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [3]:
# Create a list for holding the cities.
cities = []

# Identify nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, add it to the cities list.
    if city not in cities:
        cities.append(city)
        
len(cities)

782

In [4]:
# Import the request library.
import requests

# Import API Key from config.py file
from config import weather_api_key
# print (weather_api_key)

# Import date time to recast api date a datetime format
import time
from datetime import datetime

In [5]:
# Starting URL for Weather Map API Call.
url = f"http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID={weather_api_key}"
# print(url)

In [6]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # print(requests.get(city_url).json())
    
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Adding current_description variable
        current_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Current Description": current_description,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | ushuaia
Processing Record 2 of Set 1 | los llanos de aridane
Processing Record 3 of Set 1 | ancud
Processing Record 4 of Set 1 | esperance
Processing Record 5 of Set 1 | ciudad bolivar
Processing Record 6 of Set 1 | saskylakh
Processing Record 7 of Set 1 | barrow
Processing Record 8 of Set 1 | micheweni
Processing Record 9 of Set 1 | livingston
Processing Record 10 of Set 1 | umzimvubu
City not found. Skipping...
Processing Record 11 of Set 1 | albany
Processing Record 12 of Set 1 | attawapiskat
City not found. Skipping...
Processing Record 13 of Set 1 | rikitea
Processing Record 14 of Set 1 | bredasdorp
Processing Record 15 of Set 1 | jalu
Processing Record 16 of Set 1 | cody
Processing Record 17 of Set 1 | demyansk
Processing Record 18 of Set 1 | mataura
Processing Record 19 of Set 1 | busselton
Processing Record 20 of Set 1 | saint-philippe
Processing Record 21 of Set 1 | umm lajj
Processing R

In [7]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ushuaia,-54.8000,-68.3000,light intensity drizzle,42.46,87,75,18.41,AR,2021-12-19 20:19:08
1,Los Llanos De Aridane,28.6585,-17.9182,few clouds,65.95,81,20,4.00,ES,2021-12-19 20:22:14
2,Ancud,-41.8697,-73.8203,broken clouds,75.22,68,53,6.29,CL,2021-12-19 20:22:15
3,Esperance,-33.8667,121.9000,clear sky,58.44,69,0,2.37,AU,2021-12-19 20:22:15
4,Ciudad Bolivar,8.1222,-63.5497,overcast clouds,85.60,61,99,13.33,VE,2021-12-19 20:22:15
5,Saskylakh,71.9167,114.0833,overcast clouds,-9.72,97,100,10.85,RU,2021-12-19 20:22:16
6,Barrow,71.2906,-156.7887,overcast clouds,-2.18,71,90,21.85,US,2021-12-19 20:20:02
7,Micheweni,-4.9667,39.8333,light rain,81.54,83,17,9.44,TZ,2021-12-19 20:22:16
8,Livingston,55.9029,-3.5226,broken clouds,38.84,99,75,2.30,GB,2021-12-19 20:19:32
9,Albany,42.6001,-73.9662,broken clouds,30.45,71,63,11.01,US,2021-12-19 20:20:47


In [8]:
# reorder the columns
new_column_order = ['City', 'Country', 'Date', 'Current Description', 'Lat', 'Lng', 'Max Temp', 'Humidity', 
                    'Cloudiness', 'Wind Speed']

# Assign a new or the same DataFrame the new column order.
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Current Description,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Ushuaia,AR,2021-12-19 20:19:08,light intensity drizzle,-54.8000,-68.3000,42.46,87,75,18.41
1,Los Llanos De Aridane,ES,2021-12-19 20:22:14,few clouds,28.6585,-17.9182,65.95,81,20,4.00
2,Ancud,CL,2021-12-19 20:22:15,broken clouds,-41.8697,-73.8203,75.22,68,53,6.29
3,Esperance,AU,2021-12-19 20:22:15,clear sky,-33.8667,121.9000,58.44,69,0,2.37
4,Ciudad Bolivar,VE,2021-12-19 20:22:15,overcast clouds,8.1222,-63.5497,85.60,61,99,13.33
5,Saskylakh,RU,2021-12-19 20:22:16,overcast clouds,71.9167,114.0833,-9.72,97,100,10.85
6,Barrow,US,2021-12-19 20:20:02,overcast clouds,71.2906,-156.7887,-2.18,71,90,21.85
7,Micheweni,TZ,2021-12-19 20:22:16,light rain,-4.9667,39.8333,81.54,83,17,9.44
8,Livingston,GB,2021-12-19 20:19:32,broken clouds,55.9029,-3.5226,38.84,99,75,2.30
9,Albany,US,2021-12-19 20:20:47,broken clouds,42.6001,-73.9662,30.45,71,63,11.01


In [11]:
# Create the output file (CSV).
output_data_file = "weather_database/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")